In [ ]:
%load_ext autoreload
%autoreload 2
    
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from glob import glob
from ConvFitterVDWrapper import ConvFitterVDWrapper
from waffles.data_classes.EasyWaveformCreator import EasyWaveformCreator
from waffles.data_classes.WaveformSet import WaveformSet
from waffles.np02_utils.AutoMap import dict_endpoints_channels_list, dict_module_to_uniqch, dict_uniqch_to_module, strUch
from waffles.np02_utils.AutoMap import ordered_channels_cathode, ordered_channels_membrane, ordered_modules_cathode, ordered_modules_membrane
from waffles.np02_utils.PlotUtils import matplotlib_plot_WaveformSetGrid
from waffles.np02_utils.load_utils import ch_read_template

from utils import processfit, plot_convfit
from convfit import retrieve_responses

import mplhep
mplhep.style.use(mplhep.style.ROOT)
plt.rcParams.update({'font.size': 20,
                     'grid.linestyle': '--',
                     'axes.grid': True,
                     'figure.autolayout': True,
                     'figure.figsize': [14,6]
                     })


In [ ]:
run = 39510 

response_root_path = "/eos/experiment/neutplatform/protodune/experiments/ProtoDUNE-VD/xenon_averages/average_waveforms-hvieirad/"


response_folder_cathode = Path(response_root_path + f'run{run:06d}_cathode_response')
response_folder_membrane = Path(response_root_path + f'run{run:06d}_membrane_response')

In [ ]:
cfitparams = dict(
    threshold_align_template=0.27,
    threshold_align_response=0.2,
    error=10,
    dointerpolation=True,
    interpolation_factor=8,
    align_waveforms = True,
    dtime=16,
    convtype='fft',
    usemplhep=True,
    scinttype = "lar"
)

In [ ]:
templates = ch_read_template('templates_large_pulses')

In [ ]:
# def retrieve_responses(response_folder, output={}):
#     # file format is like this: average-endpoint-107-ch-0.txt
#     files_averages = [ Path(x) for x in glob(f"{response_folder}/*.txt")]
#     for file in files_averages:
#         fname_fragments = file.name.split('-')
#         ep = int(fname_fragments[2])
#         ch = int(fname_fragments[4].split('.')[0])
#         output[ep] = output.get(ep, {})
#         output[ep][ch] = np.loadtxt(file, dtype=np.float32)
responses = {}
counts = {}
retrieve_responses(response_folder_cathode, responses, counts)
retrieve_responses(response_folder_membrane, responses, counts)

In [ ]:
cfit = {}
dict_ep_ch_with_both_signals = {}
for ep, ch_wvf in responses.items():
    for ch, _ in ch_wvf.items():
        if ep in templates.keys():
            cfit[ep] = cfit.get(ep, {})
            if ch in templates[ep].keys():
                dict_ep_ch_with_both_signals[ep] = dict_ep_ch_with_both_signals.get(ep, []) + [ch]
                cfit[ep][ch] = ConvFitterVDWrapper(**cfitparams)

In [ ]:
dict_ok_membrane = { 107: dict_ep_ch_with_both_signals[107] }
dict_ok_cathode = { 106: dict_ep_ch_with_both_signals[106] }

In [ ]:
wfset = EasyWaveformCreator.create_WaveformSet_dictEndpointCh(dict_endpoint_ch=dict_ep_ch_with_both_signals, run_number=run)
wfsetc = EasyWaveformCreator.create_WaveformSet_dictEndpointCh(dict_endpoint_ch=dict_ok_cathode, run_number=run)
wfsetm = EasyWaveformCreator.create_WaveformSet_dictEndpointCh(dict_endpoint_ch=dict_ok_membrane, run_number=run)

In [ ]:
ep = 106
ch = 7
tmpcfit = cfit[ep][ch]
tmpcfit.set_template_waveform(templates[ep][ch])
tmpcfit.set_response_waveform(responses[ep][ch])
plt.plot(tmpcfit.template)
plt.plot(tmpcfit.response*np.max(tmpcfit.template)/np.max(tmpcfit.response))
plt.xlim(0,300)
tmpcfit.prepare_waveforms()
plt.figure()
plt.plot(tmpcfit.template)
plt.plot(tmpcfit.response*np.max(tmpcfit.template)/np.max(tmpcfit.response))

In [ ]:
tmpcfit.fit(scan=2, print_flag=False)

In [ ]:
tmpcfit.plot()
plt.legend()

In [ ]:
paramsplot = {
    'cfit':cfit,
    'dofit':False,
    'responses': responses,
    'templates': templates,
    'verbose': True,
}

matplotlib_plot_WaveformSetGrid(wfsetc, detector=ordered_modules_cathode, plot_function=plot_convfit, func_params=paramsplot, cols=4, figsize=(32,32))
# plt.savefig(f"fit_results_cathode_run{run:06d}")

In [ ]:
matplotlib_plot_WaveformSetGrid(wfsetm, detector=ordered_modules_membrane, plot_function=plot_convfit, func_params=paramsplot, cols=4, figsize=(32,32))
plt.savefig(f"fit_results_membrane_run{run:06d}")